In [0]:
!pip install -q symbulate
from symbulate import *
%matplotlib inline

# The Hypergeometric Model

The hypergeometric model is very similar to the binomial model. We still draw tickets at random from a box of the form

$$ \fbox{$\overbrace{\underbrace{\fbox{0}\ \fbox{0}\ \cdots\ \fbox{0}}_{\text{$N_0$ tickets}}\ \underbrace{\fbox{1}\ \fbox{1}\ \cdots\ \fbox{1}}_{\text{$N_1$ tickets}}}^{\text{$N$ tickets}}$}. $$

The only difference is that the draws are now made _without_ replacement. This turns out to be a useful model in many situations. For example, in quality control, a few units (e.g., cars) are randomly sampled for inspection. The inspection process may involve destroying the unit (e.g., a crash test), so it is not physically possible to inspect the same unit twice.

For concreteness, let's suppose that we randomly sample 5 cars for inspection, from a production line with 10 defective cars and 90 working cars. Letting $\fbox{1}$ represent a defective car, we can model the situation as follows:

In [0]:
model = BoxModel({0: 90, 1: 10}, size=5, replace=False)
X = RV(model, sum)
xs = X.sim(10000)
xs

We can approximate the probability of getting exactly 2 defective cars as follows:

In [0]:
xs.plot()
xs.count_eq(2) / 10000

In this section, we will derive an exact formula for probabilities in the hypergeometric model. To summarize, the **hypergeometric model** describes:

- the number of $\fbox{1}$s you get
- when you draw $n$ tickets _without_ replacement 
- from a box of $\fbox{0}$s and $\fbox{1}$s.

## A Special Case

Let's first work out the probability that we get exactly 2 $\fbox{1}$s when we draw $n=5$ tickets _without_ replacement from the box above:

$$ \fbox{$\overbrace{\underbrace{\fbox{0}\ \fbox{0}\ ...\ \fbox{0}}_{\text{$N_0 = 90$}}\ \underbrace{\fbox{1}\ \fbox{1}\ ...\ \fbox{1}}_{\text{$N_1=10$}}}^{\text{$N=100$}}$}. $$

In order to calculate probabilities, we need to work with a list of possible outcomes that are equally likely. The 100 tickets all have an equal chance of being drawn, so we imagine numbering the tickets 1 to 100 and choosing 5 at random. We might get an outcome like (51, 37, 13, 95, 60). One question is whether we should pay attention to the order of the draws. That is, should this outcome be counted as different from (13, 37, 51, 60, 95)? 

- If the order matters, then there are $100 \times 99 \times 98 \times 97 \times 96 = \frac{100!}{95!}$ equally likely outcomes.
- If the order does not matter, then there are $\binom{100}{5} = \frac{100!}{5! 95!}$ equally likely outcomes. 

We will get the same answer no matter which way we decide to count, but counting _unordered_ outcomes turns out to be easier.

How many of the $\binom{100}{5}$ unordered outcomes have exactly 2 $\fbox{1}$s?

- How many different sets of 2 $\fbox{1}$s can we choose? **Answer:** $\binom{10}{2}$
- How many different sets of 3 $\fbox{0}$s can we choose? **Answer:** $\binom{90}{3}$

These two sets can be mixed and matched in any way, resulting in $\binom{10}{2} \cdot \binom{90}{3}$ ways to end up with 2 $\fbox{1}$s.

To convert this to a probability, divide by the total number of outcomes:

$$ P(X = 2) = \frac{\binom{10}{2} \cdot \binom{90}{3}}{\binom{100}{5}}. $$

In [0]:
def choose(n, k):
    return factorial(n) / (factorial(k) * factorial(n - k))

(choose(10, 2) * choose(90, 3)) / choose(100, 5)

## The General Case

Now let's extend the derivation above to general $n$, $N_1$, and $N_0$. (Recall that in the example above, $n = 5$, $N_1 = 10$ and $N_0 = 90$.) The probability of getting exactly $x$ $\fbox{1}$s is 

$$ P(X = x) = \frac{\binom{N_1}{x} \binom{N_0}{n-x}}{\binom{N}{n}}. $$

Unlike the binomial model, the hypergeometric model depends on the exact values of $N_0$ and $N_1$, not just their relative values. Think about why this makes intuitive sense. Remember that in the hypergeometric model, draws are made _without_ replacement. What would have happened if we had started with only $N=10$ tickets in the above example?

Once you specify the hypergeometric model, you can use Symbulate to evaluate the probability:

In [0]:
Hypergeometric(n=5, N1=10, N0=90).pdf(2)

## Example. Capture-Recapture

How do we estimate the population of fish in a lake? It is infeasible to try to catch every fish. One way is the method of capture-recapture. First, we capture some number of fish---say, 10 fish---tag them and release them back into the lake. Then, we wait, giving the tagged fish a chance to mix back into the general population. A week later, we catch another 20 fish, some of which will have been tagged the previous week. (Note that we catch all 20 fish before releasing them back into lake so no fish can be caught twice.) The proportion of tagged fish in this second catch can be used to estimate the population of fish in the pond.

To be precise, let $X$ be the number of tagged fish in the second catch and $N$ be the total number of fish in the pond. Then, the proportion of fish in the second catch that is tagged ($X / 20$) should be approximately equal to the overall proportion of tagged fish in the pond ($10 / N$): 

$$ \frac{X}{20} \approx \frac{10}{N}. $$

This suggests the estimator 

$$ \hat N = \frac{10 \cdot 20}{X} $$

for the number of fish in the pond. We will study the properties of this estimator later in this course. 

For now, we study the following _probability question_: suppose there are actually $N = 50$ fish in the pond, and we carry out the procedure described above. What is the probability that fewer than 4 fish (in the second catch) are tagged? In other words, what is $P(X < 4)$?

In [0]:
# YOUR CODE HERE